In [ ]:
# streamlit_app.py
# A non-text-based Python UI for your Blackjack + MC Control agent using Streamlit.
# Run with:
#   pip install streamlit
#   streamlit run streamlit_app.py
import json
import random
import time
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

import streamlit as st

# -----------------------------
# ----- Blackjack environment
# -----------------------------

def draw_card():
    # Uniform over 1..10 with 10 appearing 4x as often (10, J, Q, K)
    cards = [1,2,3,4,5,6,7,8,9,10,10,10,10]
    return random.choice(cards)

def draw_hand():
    return [draw_card(), draw_card()]

def usable_ace(hand: List[int]) -> bool:
    return 1 in hand and sum(hand) + 10 <= 21

def hand_value(hand: List[int]) -> int:
    val = sum(hand)
    return val + 10 if usable_ace(hand) else val

def is_natural(hand: List[int]) -> bool:
    return sorted(hand) == [1, 10]

@dataclass
class StepResult:
    state: Tuple[int, int, bool]
    reward: int
    done: bool
    info: Dict

class BlackjackEnv:
    # Actions: 0 = stick, 1 = hit
    def __init__(self, hit_on_soft_17: bool = False):
        self.hit_on_soft_17 = hit_on_soft_17
        self.reset()

    def reset(self) -> Tuple[int, int, bool]:
        self.player = draw_hand()
        self.dealer = draw_hand()
        if is_natural(self.player) or is_natural(self.dealer):
            self.done = True
            self.reward = self._natural_outcome()
        else:
            self.done = False
            self.reward = 0
        return self._obs()

    def _natural_outcome(self) -> int:
        p_nat = is_natural(self.player)
        d_nat = is_natural(self.dealer)
        if p_nat and not d_nat:
            return 1
        if d_nat and not p_nat:
            return -1
        return 0

    def _dealer_policy(self, hand: List[int]):
        while True:
            val = hand_value(hand)
            if val < 17:
                hand.append(draw_card())
                continue
            if val == 17 and usable_ace(hand) and self.hit_on_soft_17:
                hand.append(draw_card())
                continue
            break

    def step(self, action: int) -> StepResult:
        if self.done:
            return StepResult(self._obs(), self.reward, self.done, {})

        if action == 1:  # hit
            self.player.append(draw_card())
            if hand_value(self.player) > 21:
                self.done = True
                self.reward = -1
            return StepResult(self._obs(), self.reward, self.done, {})

        # stick
        self._dealer_policy(self.dealer)
        self.done = True
        p, d = hand_value(self.player), hand_value(self.dealer)
        if d > 21 or p > d:
            self.reward = 1
        elif p < d:
            self.reward = -1
        else:
            self.reward = 0
        return StepResult(self._obs(), self.reward, self.done, {})

    def _obs(self) -> Tuple[int, int, bool]:
        return (hand_value(self.player), self.dealer[0], usable_ace(self.player))

# -----------------------------
# ----- Monte Carlo Control
# -----------------------------
from collections import defaultdict

class MCBlackjackAgent:
    # State key: (player_sum, dealer_up, usable_ace) ; Actions: 0 stick, 1 hit
    def __init__(self, epsilon: float = 0.1, gamma: float = 1.0):
        self.epsilon = epsilon
        self.gamma = gamma
        self.Q = defaultdict(lambda: [0.0, 0.0])
        self.returns_sum = defaultdict(lambda: [0.0, 0.0])
        self.returns_count = defaultdict(lambda: [0, 0])

    def policy(self, state: Tuple[int, int, bool]) -> int:
        import random as _r
        if _r.random() < self.epsilon:
            return _r.choice([0, 1])
        q = self.Q[state]
        return 0 if q[0] >= q[1] else 1

    def greedy_action(self, state: Tuple[int, int, bool]) -> int:
        q = self.Q[state]
        return 0 if q[0] >= q[1] else 1

    def generate_episode(self, env: BlackjackEnv):
        episode = []
        state = env.reset()
        if env.done:
            return [(state, None, env.reward)]
        while True:
            a = self.policy(state)
            sr = env.step(a)
            episode.append((state, a, sr.reward))
            state = sr.state
            if sr.done:
                break
        return episode

    def train(self, env_factory, episodes: int = 100000, progress_cb=None):
        for i in range(episodes):
            env = env_factory()
            ep = self.generate_episode(env)
            visited = set()
            G = 0.0
            for t in reversed(range(len(ep))):
                s, a, r = ep[t]
                G = self.gamma * G + r
                if a is None:
                    continue
                sa = (s, a)
                if sa not in visited:
                    visited.add(sa)
                    self.returns_sum[s][a] += G
                    self.returns_count[s][a] += 1
                    self.Q[s][a] = self.returns_sum[s][a] / self.returns_count[s][a]
            if progress_cb and (i + 1) % 1000 == 0:
                progress_cb(i + 1)

    def export_q(self) -> Dict[str, List[float]]:
        # Serialize keys as strings for JSON
        out = {}
        for (ps, up, ua), q in self.Q.items():
            out[f"{ps}|{up}|{1 if ua else 0}"] = q
        return out

    def import_q(self, data: Dict[str, List[float]]):
        self.Q.clear()
        for k, v in data.items():
            ps, up, ua = k.split("|")
            self.Q[(int(ps), int(up), bool(int(ua)))] = v

# -----------------------------
# ----- UI helpers
# -----------------------------
SUITS = ["♠", "♥", "♦", "♣"]
RANKS_FOR_10 = ["10", "J", "Q", "K"]

def to_card_face(v: int) -> str:
    if v == 1:
        rank = "A"
    elif v == 10:
        rank = random.choice(RANKS_FOR_10)
    else:
        rank = str(v)
    suit = random.choice(SUITS)
    return f"{rank}{suit}"

CARD_CSS = """
<style>
.card {display:inline-flex; align-items:center; justify-content:center; width:64px; height:96px; margin-right:8px; border-radius:12px; border:1px solid rgba(0,0,0,0.15); background:white; font-weight:700; font-size:22px; box-shadow: 0 2px 8px rgba(0,0,0,0.08)}
.hidden {background: #0b132b; color:#0b132b;}
.pill {display:inline-flex; align-items:center; padding:2px 8px; border-radius:999px; border:1px solid rgba(0,0,0,0.1); background:rgba(0,0,0,0.04); font-size:12px;}
</style>
"""

# -----------------------------
# ----- Streamlit App
# -----------------------------
st.set_page_config(page_title="Blackjack MC (Python)", page_icon="🂡", layout="wide")
st.markdown(CARD_CSS, unsafe_allow_html=True)

# Seed for reproducibility (optional)
random.seed(1)

# Session state init
if "env" not in st.session_state:
    st.session_state.env = BlackjackEnv(hit_on_soft_17=False)
    st.session_state.state = st.session_state.env._obs()
    st.session_state.done = st.session_state.env.done
    st.session_state.reward = st.session_state.env.reward
    st.session_state.reveal = st.session_state.env.done

if "agent" not in st.session_state:
    st.session_state.agent = MCBlackjackAgent(epsilon=0.1, gamma=1.0)

if "stats" not in st.session_state:
    st.session_state.stats = {"wins": 0, "losses": 0, "pushes": 0}

# Sidebar controls
st.sidebar.header("Dealer Rules")
hit_soft_17 = st.sidebar.toggle("Dealer hits on soft 17", value=st.session_state.env.hit_on_soft_17)
if hit_soft_17 != st.session_state.env.hit_on_soft_17:
    st.session_state.env = BlackjackEnv(hit_on_soft_17=hit_soft_17)
    s = st.session_state.env.reset()
    st.session_state.state = s
    st.session_state.done = st.session_state.env.done
    st.session_state.reward = st.session_state.env.reward
    st.session_state.reveal = st.session_state.env.done

st.sidebar.header("MC Agent Training")
epsilon = st.sidebar.slider("ε (exploration)", 0.0, 0.5, float(st.session_state.agent.epsilon), 0.01)
st.session_state.agent.epsilon = epsilon
episodes = st.sidebar.slider("Episodes", 1_000, 200_000, 50_000, 1_000)

col_sb1, col_sb2 = st.sidebar.columns(2)
train_clicked = col_sb1.button("Train", use_container_width=True)
reset_stats = col_sb2.button("Reset W/L/P", use_container_width=True)

if reset_stats:
    st.session_state.stats = {"wins": 0, "losses": 0, "pushes": 0}

# Save/Load
st.sidebar.subheader("Policy I/O")
col_io1, col_io2 = st.sidebar.columns([1,1])
if col_io1.button("Save Q-table", use_container_width=True):
    data = json.dumps(st.session_state.agent.export_q(), indent=2)
    st.sidebar.download_button("Download JSON", data, file_name="blackjack_mc_policy.json", mime="application/json")

uploaded = st.sidebar.file_uploader("Load Q-table JSON", type=["json"])
if uploaded is not None:
    try:
        data = json.load(uploaded)
        st.session_state.agent.import_q(data)
        st.sidebar.success("Loaded policy ✔")
    except Exception as e:
        st.sidebar.error(f"Failed to load: {e}")

# Autoplay
st.sidebar.header("Autoplay (agent plays)")
auto_hands = st.sidebar.slider("Hands to play", 1, 200, 20, 1)
speed_ms = st.sidebar.slider("Speed (ms/step)", 0, 1000, 300, 25)
autoplay = st.sidebar.button("Start Autoplay", use_container_width=True)

# Main Header
st.title("Blackjack Monte Carlo — Python UI (Streamlit)")
st.caption("Learn & play Blackjack with your Monte Carlo control agent — no terminal required ✨")

# Training action
if train_clicked:
    progress = st.progress(0, text="Training agent…")
    trained = 0
    batch = 10_000
    def cb(done):
        pass  # optional hook per 1000 eps
    while trained < episodes:
        step = min(batch, episodes - trained)
        st.session_state.agent.train(lambda: BlackjackEnv(hit_on_soft_17=st.session_state.env.hit_on_soft_17), episodes=step, progress_cb=cb)
        trained += step
        progress.progress(int(100 * trained / episodes), text=f"Training agent… {trained:,}/{episodes:,}")
        # brief yield for UI
        time.sleep(0.01)
    progress.empty()
    st.success(f"Training complete: {episodes:,} episodes")

# Board layout
left, right = st.columns([2, 1])

with left:
    # Dealer hand
    st.subheader("Dealer")
    reveal = st.session_state.reveal
    dealer = st.session_state.env.dealer
    player = st.session_state.env.player

    dcards = []
    for i, v in enumerate(dealer):
        hidden = (not reveal) and i == 1
        label = "" if hidden else to_card_face(v)
        klass = "card hidden" if hidden else "card"
        dcards.append(f'<div class="{klass}">{label}</div>')
    st.markdown("".join(dcards), unsafe_allow_html=True)

    # Player hand
    st.subheader("Player")
    pcards = [f'<div class="card">{to_card_face(v)}</div>' for v in player]
    st.markdown("".join(pcards), unsafe_allow_html=True)

    # Info row
    ps, up, ua = st.session_state.state
    status_cols = st.columns(4)
    with status_cols[0]:
        st.metric("Player Sum", ps)
    with status_cols[1]:
        st.metric("Dealer Upcard", up)
    with status_cols[2]:
        st.metric("Usable Ace", "Yes" if ua else "No")
    with status_cols[3]:
        if st.session_state.done:
            res = "WIN" if st.session_state.reward > 0 else ("LOSS" if st.session_state.reward < 0 else "PUSH")
            st.metric("Result", res)
        else:
            sug = "Stick" if st.session_state.agent.greedy_action(st.session_state.state) == 0 else "Hit"
            st.metric("MC Suggests", sug)

    # Action buttons
    a1, a2, a3 = st.columns(3)
    if a1.button("Hit", type="primary", use_container_width=True, disabled=st.session_state.done):
        sr = st.session_state.env.step(1)
        st.session_state.state = sr.state
        st.session_state.reward = sr.reward
        st.session_state.done = sr.done
        if sr.done:
            st.session_state.reveal = True
            if sr.reward > 0:
                st.session_state.stats["wins"] += 1
            elif sr.reward < 0:
                st.session_state.stats["losses"] += 1
            else:
                st.session_state.stats["pushes"] += 1
        st.rerun()

    if a2.button("Stick", use_container_width=True, disabled=st.session_state.done):
        sr = st.session_state.env.step(0)
        st.session_state.state = sr.state
        st.session_state.reward = sr.reward
        st.session_state.done = sr.done
        if sr.done:
            st.session_state.reveal = True
            if sr.reward > 0:
                st.session_state.stats["wins"] += 1
            elif sr.reward < 0:
                st.session_state.stats["losses"] += 1
            else:
                st.session_state.stats["pushes"] += 1
        st.rerun()

    if a3.button("New Hand", use_container_width=True):
        s = st.session_state.env.reset()
        st.session_state.state = s
        st.session_state.done = st.session_state.env.done
        st.session_state.reward = st.session_state.env.reward
        st.session_state.reveal = st.session_state.env.done
        # naturals resolve immediately; update stats
        if st.session_state.done:
            r = st.session_state.reward
            if r > 0:
                st.session_state.stats["wins"] += 1
            elif r < 0:
                st.session_state.stats["losses"] += 1
            else:
                st.session_state.stats["pushes"] += 1
        st.rerun()

with right:
    st.subheader("Stats")
    c1, c2, c3 = st.columns(3)
    c1.metric("Wins", st.session_state.stats["wins"])
    c2.metric("Losses", st.session_state.stats["losses"])
    c3.metric("Pushes", st.session_state.stats["pushes"])

    st.divider()
    st.subheader("Autoplay (Greedy Policy)")
    if autoplay:
        progress = st.progress(0, text="Playing hands…")
        for i in range(auto_hands):
            s = st.session_state.env.reset()
            st.session_state.state = s
            st.session_state.done = st.session_state.env.done
            st.session_state.reward = st.session_state.env.reward
            st.session_state.reveal = st.session_state.env.done

            if st.session_state.done:
                # natural resolution
                r = st.session_state.reward
                if r > 0:
                    st.session_state.stats["wins"] += 1
                elif r < 0:
                    st.session_state.stats["losses"] += 1
                else:
                    st.session_state.stats["pushes"] += 1
            else:
                while not st.session_state.env.done:
                    a = st.session_state.agent.greedy_action(st.session_state.env._obs())
                    sr = st.session_state.env.step(a)
                    st.session_state.state = sr.state
                    st.session_state.reward = sr.reward
                    st.session_state.done = sr.done
                    time.sleep(max(0, speed_ms) / 1000.0)
                st.session_state.reveal = True
                r = st.session_state.reward
                if r > 0:
                    st.session_state.stats["wins"] += 1
                elif r < 0:
                    st.session_state.stats["losses"] += 1
                else:
                    st.session_state.stats["pushes"] += 1
            progress.progress(int(100 * (i + 1) / auto_hands), text=f"Playing hands… {i+1}/{auto_hands}")
        progress.empty()
        st.success("Autoplay finished")
        st.rerun()

st.info("""
**How to run**
1. Install Streamlit: `pip install streamlit`
2. Save this file as `streamlit_app.py`.
3. Start the app: `streamlit run streamlit_app.py`.

This uses your exact environment and Monte Carlo control logic, with graphical cards, buttons, training, autoplay, and save/load of the Q-table.
""")


  Using cached streamlit-1.51.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached protobuf-6.33.0-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached narwhals-2.10.1-py3-none-any.whl.metadata (11 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-macosx_10_13_u

2025-11-03 11:28:21.836 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 11:28:21.837 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 11:28:21.870 
  command:

    streamlit run /Users/williamlonon/.venvs/default/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-03 11:28:21.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 11:28:21.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 11:28:21.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 11:28:21.872 Session state does not function when running a script without `

DeltaGenerator()